In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from transformers import ViTForImageClassification, ViTImageProcessor
from tqdm import tqdm
from PIL import Image
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    confusion_matrix, 
    roc_curve, 
    auc, 
    roc_auc_score, 
    classification_report
)

torch.cuda.empty_cache()

# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
batch_size = 16
num_epochs = 100
learning_rate = 1e-4
weight_decay = 1e-2
lasso_lambda = 1e-3  # Lasso regularization strength

# Load pre-trained ViT model and image processor
model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
image_processor = ViTImageProcessor.from_pretrained(model_name)

# Replace the classifier head with a slightly more complex one
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(model.config.hidden_size, 1024),
    torch.nn.BatchNorm1d(1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(1024, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 2)
)

# Define data transforms with augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess the datasets
train_dir = '/Users/user/Downloads/archive/IMG_CLASSES/train3'
val_dir = '/Users/user/Downloads/archive/IMG_CLASSES/val3'

train_dataset = ImageFolder(train_dir, transform=train_transform)
val_dataset = ImageFolder(val_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4)

# Move model to device
model.to(device)

# Custom loss function with Lasso regularization
def lasso_loss(model, criterion, outputs, labels, lasso_lambda):
    # Standard cross-entropy loss
    standard_loss = criterion(outputs.logits, labels)
    
    # Lasso regularization (L1 penalty)
    l1_penalty = sum(p.abs().sum() for p in model.classifier.parameters() if p.requires_grad)
    
    # Combined loss
    return standard_loss + lasso_lambda * l1_penalty

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = torch.nn.CrossEntropyLoss()

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, epochs=num_epochs, steps_per_epoch=len(train_loader))

# Lists to store training and validation metrics
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
val_precisions = []
val_recalls = []
val_f1_scores = []

# Best model tracking
best_val_accuracy = 0
best_model_path = 'best_vit_image_classifier.pth'

# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        data = data.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        outputs = model(data, labels=label)
        
        # Use custom loss with Lasso regularization
        loss = lasso_loss(model, criterion, outputs, label, lasso_lambda)

        loss.backward()
        optimizer.step()
        scheduler.step()

        acc = (outputs.logits.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc.item() / len(train_loader)
        epoch_loss += loss.item() / len(train_loader)

    # Store training metrics
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    # Validation
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    epoch_val_loss = 0

    with torch.no_grad():
        for data, label in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            data = data.to(device)
            label = label.to(device)

            outputs = model(data, labels=label)
            val_loss = criterion(outputs.logits, label)

            # Collect predictions and probabilities
            probs = torch.nn.functional.softmax(outputs.logits, dim=1)
            preds = torch.argmax(probs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())

            epoch_val_loss += val_loss.item() / len(val_loader)

    # Calculate validation metrics
    val_accuracy = accuracy_score(all_labels, all_preds)
    val_precision = precision_score(all_labels, all_preds, average='weighted')
    val_recall = recall_score(all_labels, all_preds, average='weighted')
    val_f1 = f1_score(all_labels, all_preds, average='weighted')

    # Save the best model based on validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_accuracy': val_accuracy,
        }, best_model_path)
        print(f"\nBest model saved with validation accuracy: {best_val_accuracy:.4f}")

    # Store validation metrics
    val_losses.append(epoch_val_loss)
    val_accuracies.append(val_accuracy)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_f1_scores.append(val_f1)

    print(
        f"Epoch : {epoch+1} - "
        f"train_loss : {epoch_loss:.4f} - train_acc: {epoch_accuracy:.4f} - "
        f"val_loss : {epoch_val_loss:.4f} - val_acc: {val_accuracy:.4f}\n"
        f"Validation Metrics: "
        f"Precision: {val_precision:.4f}, "
        f"Recall: {val_recall:.4f}, "
        f"F1-Score: {val_f1:.4f}\n"
    )

# Rest of the code remains the same...

# Add a note about model loading
print("\nTo load the best saved model:")
print("checkpoint = torch.load('vit.pth')")
print("model.load_state_dict(checkpoint['model_state_dict'])")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/100 - Validation: 100%|████████████████████████████████████████████████████████| 63/63 [00:29<00:00,  2.13it/s]



Best model saved with validation accuracy: 0.9260
Epoch : 1 - train_loss : 23.9294 - train_acc: 0.8564 - val_loss : 0.2252 - val_acc: 0.9260
Validation Metrics: Precision: 0.9268, Recall: 0.9260, F1-Score: 0.9260



Epoch 2/100 - Validation: 100%|████████████████████████████████████████████████████████| 63/63 [00:29<00:00,  2.13it/s]



Best model saved with validation accuracy: 0.9560
Epoch : 2 - train_loss : 22.7334 - train_acc: 0.9586 - val_loss : 0.1376 - val_acc: 0.9560
Validation Metrics: Precision: 0.9560, Recall: 0.9560, F1-Score: 0.9560



Epoch 3/100 - Validation: 100%|████████████████████████████████████████████████████████| 63/63 [00:29<00:00,  2.15it/s]


...

Epoch : 98 - train_loss : 0.0646 - train_acc: 0.9998 - val_loss : 0.0086 - val_acc: 0.9990
Validation Metrics: Precision: 0.9990, Recall: 0.9990, F1-Score: 0.9990



Epoch 99/100 - Validation: 100%|███████████████████████████████████████████████████████| 63/63 [00:29<00:00,  2.14it/s]


Epoch : 99 - train_loss : 0.0646 - train_acc: 0.9998 - val_loss : 0.0098 - val_acc: 0.9990
Validation Metrics: Precision: 0.9990, Recall: 0.9990, F1-Score: 0.9990



Epoch 100/100 - Validation: 100%|██████████████████████████████████████████████████████| 63/63 [00:29<00:00,  2.15it/s]

Epoch : 100 - train_loss : 0.0720 - train_acc: 0.9969 - val_loss : 0.0088 - val_acc: 0.9990
Validation Metrics: Precision: 0.9990, Recall: 0.9990, F1-Score: 0.9990


To load the best saved model:
checkpoint = torch.load('vit.pth')
model.load_state_dict(checkpoint['model_state_dict'])
